In [1]:
import numpy as np
import pandas as pd
from cmdstanpy import CmdStanModel
from itertools import islice

# Load data

In [2]:
df = pd.read_csv("data/data-shogi-player.txt")
print(len(df))
df.head(5)

6231


,Loser,Winner
0,22,83
1,39,27
2,47,105
3,1,7
4,45,81


# Run Stan

In [3]:
model = CmdStanModel(stan_file="model/model10-4.stan")

INFO:cmdstanpy:compiling stan program, exe file: /workdir/chapter10/model/model10-4
INFO:cmdstanpy:compiler options: stanc_options=None, cpp_options=None
INFO:cmdstanpy:compiled model file: /workdir/chapter10/model/model10-4


In [4]:
set_players = set().union(df["Winner"].values).union(df["Loser"].values)
len(set_players)

166

In [5]:
data = {
    "N": len(set_players),
    "G": len(df),
    "LW": df.values
}

In [6]:
fit = model.sample(data=data, show_progress=True)

Chain 2 - warmup:   0%|          | 0/1 [00:00<?, ?it/s]

Chain 2 - warmup:   0%|          | 0/1 [00:00<?, ?it/s]


Chain 4 - warmup:   0%|          | 0/1 [00:00<?, ?it/s]

Chain 3 - warmup:   0%|          | 0/1 [00:00<?, ?it/s]


Chain 2 - warmup:   0%|          | 0/2000 [00:00<?, ?it/s]

Chain 3 - warmup:   0%|          | 0/2000 [00:00<?, ?it/s]


Chain 4 - warmup:   0%|          | 0/2000 [00:00<?, ?it/s]

Chain 2 - warmup:   5%|▌         | 100/2000 [01:14<23:43,  1.34it/s]


Chain 1 - warmup:   5%|▌         | 100/2000 [01:18<24:54,  1.27it/s]


Chain 4 - warmup:  10%|█         | 200/2000 [02:12<21:09,  1.42it/s]

Chain 1 - warmup:  10%|█         | 200/2000 [02:23<22:20,  1.34it/s]


Chain 4 - warmup:  15%|█▌        | 300/2000 [03:05<18:26,  1.54it/s]

Chain 2 - warmup:  20%|██        | 400/2000 [03:50<15:30,  1.72it/s]

Chain 3 - warmup:  20%|██        | 400/2000 [03:51<15:16,  1.75it/s]


Chain 2 - warmup:  25%|██▌       | 500/2000 [04:36<13:39,  1.83it/s]

Chain 3 - warmup:  25%|██

In [11]:
summary = fit.summary()
summary[summary["R_hat"] >= 1.1]

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
name,,,,,,,,,


# Show result

In [7]:
players = dict()
with open("data/data-shogi-player-name.txt") as f:
    for line in islice(f, 1, None):
        tmp = line.strip().split("\t")
        players[int(tmp[2])] = tmp[1]

In [8]:
pf = np.median(fit.stan_variable("mu"), axis=0)
s_pf = np.median(fit.stan_variable("s_pf"), axis=0)

In [9]:
for i in islice(np.argsort(-pf), 0, 10):
    print(i+1, players[i+1], pf[i], s_pf[i])

47 羽生善治 1.863785 0.836843
105 渡辺明 1.610375 1.0213299999999998
134 豊島将之 1.33272 0.975645
78 行方尚史 1.29928 0.7833034999999999
65 郷田真隆 1.2862749999999998 0.9419329999999999
130 糸谷哲郎 1.25749 1.225095
52 佐藤康光 1.252625 0.989267
53 森内俊之 1.17283 1.2390850000000002
77 久保利明 1.1164100000000001 0.857098
59 屋敷伸之 1.1016249999999999 0.7951945


In [10]:
for i in islice(np.argsort(-s_pf), 0, 10):
    print(i+1, players[i+1], s_pf[i])

155 藤森哲也 1.2644950000000001
53 森内俊之 1.2390850000000002
130 糸谷哲郎 1.225095
86 松本佳介 1.20546
99 山本真也 1.19767
71 深浦康市 1.193835
116 横山泰明 1.1872500000000001
51 神崎健二 1.15542
140 田中悠一 1.152065
109 橋本崇載 1.151125
